# 2.3.2.2 Textlicher Teil Bebauungsplan – Embeddings + Vector Store (GPT-4o)

Das Context-Window (128.000 Tokens) in GPT-4o ist begrenzt. Entsprechend muss ab einer gewissen Seitenanzahl des schriftlichen Dokuments eines Bebauungsplans auf Emebddings zurückgegriffen werden. (~765 Tokens pro Bild mit hoher Auflösung / 85 Tokens pro Bild mit niedriger Auflösung)

* Zero-Shot
* Chain-Of-Thought

In [63]:
import asyncio
import os
from dotenv import load_dotenv, find_dotenv
from dotmap import DotMap
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_chroma import Chroma
from utils.parser import Parser
from utils.openai import OpenAI
from utils.runner import Runner
from utils.pprint import pprint

instructions =  "Du bist ein Assistent zur getreuen Wiedergabe von Textinformationen aus Dokumenten. Achte auf Vollständigkeit."
ava = OpenAI(instructions)
parser = Parser()
runner = Runner()

load_dotenv(find_dotenv())
embeddings_model = OpenAIEmbeddings(
    api_key=os.getenv("OPENAI_API_KEY"),
    model="text-embedding-3-large",
)

In [ ]:
# A) OCR via RapidOCR --> CNN/RNN != Transformer (LLM)
# from langchain_community.document_loaders import PyMuPDFLoader
# pdf_path = "../data/raw/bpläne/2_zeichnung_textteil_getrennt/F 11- 02 Gewerbegebiet Himmelreich - schriftlicher Teil.pdf"
# loader = PyMuPDFLoader(pdf_path, extract_images=True)
# data = loader.load()
# print(data[0])

In [64]:
# B1) OCR via GPT-4o / Kompletter schriftlicher Teil
# Idee: Jede Seite einzeln verarbeiten, um den Fokus zu maximieren und Context-Window klein zu halten.
async def page2text(page_prompt):
    msg = ava.request([
        {
            "type": "text",
            "text": 'Extrahiere den kompletten Textinhalt. Output im LaTeX-Format.'
        },
        page_prompt
    ])
    return msg

pdf_path = "../data/raw/bpläne/2_zeichnung_textteil_getrennt/F11-01-TT.pdf"
prompts = parser.pdf2prompts(pdf_path)
prompt_chain = list(map(lambda prompt: page2text(prompt), prompts))
data_list = await asyncio.gather(*prompt_chain)
pprint(data_list)

```latex
\documentclass{article}
\usepackage[utf8]{inputenc}

\begin{document}

\section*{SCHRIFTLICHER TEIL (Teil B)}

\subsection*{BEBBAUUNGSPLAN "GEWERBEGEBIET HIMMELREICH"}

\subsubsection*{STADT LAICHINGEN, GEMARKUNG FELDSTETTEN, ALB-DONAU-KREIS}

Der Geltungsbereich wird durch das Planzeichen im Lageplan begrenzt.

Lageplan M 1: 500

Für die planungsrechtlichen bzw. bauordnungsrechtlichen Festsetzungen gelten:

\begin{itemize}
    \item \textbf{Baugesetzbuch (BauGB)}\\
    in der Fassung der Bekanntmachung vom 27.08.1997 (BGBl. I. S. 2141).
    
    \item \textbf{Baunutzungsverordnung (BauNVO)}\\
    in der Fassung der Bekanntmachung vom 23.01.1990 (BGBl. S. 132), zuletzt geändert am 22.04.1993 (BGBl. I. S. 466).
    
    \item \textbf{Planzeichenverordnung 1990 (PlanZV 90)}\\
    in der Fassung der Bekanntmachung vom 18.12.1990 (BGBl. I. S. 58).
    
    \item \textbf{Landesbauordnung (LBO)}\\
    in der Fassung der Bekanntmachung vom 08.08.1995 (GBl. S. 617).
\end{itemize}

\su

In [65]:
# B2) OCR via GPT-4o / Kompletter schriflitcher Teil
# Idee: Komplettes PDF auf einmal einlesen, sodass Layout konsistent bleibt.
pdf_path = "../data/raw/bpläne/2_zeichnung_textteil_getrennt/F11-01-TT.pdf"
pdf_prompts = parser.pdf2prompts(pdf_path)

pdf_text = ava.request([
    *pdf_prompts,
    {
        "type": "text",
        "text": 'Extrahiere den kompletten Textinhalt. Output im LaTeX-Format.'
    },
])

pprint(pdf_text)

```latex
\documentclass{article}
\usepackage[utf8]{inputenc}
\usepackage{graphicx}

\begin{document}

\section*{Schriftlicher Teil (Teil B)}
\textbf{Bebauungsplan "Gewerbegebiet Himmelreich"} \\
Stadt Laichingen, Gemarkung Feldstetten, Alb-Donau-Kreis

\subsection*{}
Der Geltungsbereich wird durch das Planzeichen im Lageplan begrenzt. \\
Lageplan M 1: 500

\subsection*{}
Für die planungsrechtlichen bzw. bauordnungsrechtlichen Festsetzungen gelten:
\begin{itemize}
    \item Baugesetzbuch (BauGB) in der Fassung der Bekanntmachung vom 27.08.1997 (BGBl. I. S. 2141).
    \item Baunutzungsverordnung (BauNVO) in der Fassung der Bekanntmachung vom 23.01.1990 (BGBl. S. 132), zuletzt geändert am 22.04.1993 (BGBl. I. S. 466).
    \item Planzeichenverordnung 1990 (PlanZV 90) in der Fassung der Bekanntmachung vom 18.12.1990 (BGBl. I. S. 58).
    \item Landesbauordnung (LBO) in der Fassung der Bekanntmachung vom 08.08.1995 (GBl. S. 617).
\end{itemize}

\subsection*{Bisherige Festsetzungen:}
Mit in K

In [69]:
# C1) Ein Thread
# Nachteil: Visueller Kontext geht verloren
text_prompts = parser.text2prompts(data_list)
msg = ava.request([
    *text_prompts,
        {
        "type": "text",
        "text": 'Extrahiere alle Informationen zu den folgenden Themen: Art der baulichen Nutzung, Maß der baulichen Nutzung, Bauweise, überbaubare Grundstücksfläche.'
    }
])

pprint(msg)

Hier sind die extrahierten Informationen zu den Themen "Art der baulichen Nutzung", "Maß der baulichen Nutzung", "Bauweise" und "überbaubare Grundstücksfläche":

### Art der baulichen Nutzung
#### Eingeschränktes Gewerbegebiet (GEE) (§ 8 BauNVO i.V.m. § 1 (4) BauNVO)
- **Zulässig sind:**
  - Gewerbebetriebe, die das Wohnen nicht wesentlich stören, wobei bei Einzelhandelsbetrieben nur die nicht innenstadtrelevanten Sortimente (Kfz, Motorräder, Mopeds, Fahrräder, Kfz-Zubehör, Rasenmäher, Landmaschinen, Fahrrad- und Motorradzubehör, Brennstoffe und Mineralölerzeugnisse) zulässig sind.
  - Der Verkauf von auf dem Grundstück produzierten Waren auf einer untergeordneten Fläche ist zulässig.
- **Nicht zulässig sind:**
  - Nutzungen gemäß § 8 (3) Nr. 3 BauNVO (Vergnügungsstätten) sind nicht zulässig.
  - Einzelhandelsbetriebe mit innenstadtrelevanten und bestimmten nicht innenstadtrelevanten Sortimenten sind nicht zulässig.

#### Gewerbegebiet (GE) (§ 8 BauNVO)
- **Zulässig sind:**
  - Gewerbe

In [70]:
# C2) Mehrere Threads – Extrahierter Text
# Nachteile: 
# * Visueller Kontext geht verloren
# * Fehlender Kontext zwischen den Chunks
# * Lösung enthält exkterne Logik via Vektor-Store
async def run(page_content):
    msg = ava.request([
        {
            "type": "text",
            "text": 'Extrahiere alle Informationen zu den folgenden Themen: Art der baulichen Nutzung, Maß der baulichen Nutzung, Bauweise, überbaubare Grundstücksfläche.'
        },
        {
            "type": "text",
            "text": page_content
        },
    ])
    return msg

prompt_chain = list(map(lambda page_content: run(page_content), data_list))
results = await asyncio.gather(*prompt_chain)
pprint(results)

Der bereitgestellte Text enthält keine spezifischen Informationen zu den Themen "Art der baulichen Nutzung", "Maß der baulichen Nutzung", "Bauweise" und "überbaubare Grundstücksfläche". Der Text beschreibt lediglich die rechtlichen Grundlagen und die Geltungsbereiche des Bebauungsplans "Gewerbegebiet Himmelreich" in der Stadt Laichingen, Gemarkung Feldstetten, Alb-Donau-Kreis.

Um die gewünschten Informationen zu extrahieren, müssten detaillierte Festsetzungen oder Regelungen zu diesen Themen im Bebauungsplan enthalten sein. Da diese Informationen im bereitgestellten Text fehlen, kann keine Extraktion erfolgen.
#############################################
Hier sind die extrahierten Informationen zu den Themen "Art der baulichen Nutzung", "Maß der baulichen Nutzung", "Bauweise" und "überbaubare Grundstücksfläche" aus dem bereitgestellten Dokument:

### Art der baulichen Nutzung
#### Eingeschränktes Gewerbegebiet (GEE) (§ 8 BauNVO i.V.m. § 1 (4) BauNVO)
- **Zulässig sind:**
  - Gewerbeb

In [71]:
# C3) Vector Store
# Nachteile:
# * Visueller Kontext geht verloren.
# * Größe von k schränkte vollständige Extraktion ein.
documents = list(map(lambda item: DotMap({"page_content":item[1], "metadata": DotMap({"page": item[0]+1})}), enumerate(data_list)))
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000)
splits = text_splitter.split_documents(documents)
vectorstore = Chroma()
vectorstore.reset_collection()
vectorstore = Chroma.from_documents(documents=splits, embedding=embeddings_model)
query = 'Extrahiere alle Informationen zu den folgenden Themen: Art der baulichen Nutzung, Maß der baulichen Nutzung, Bauweise, überbaubare Grundstücksfläche.'
retrieved_docs = vectorstore.similarity_search_with_relevance_scores(query, k=25)
retrieved_text = list(map(lambda item: item[0].page_content, retrieved_docs))
text_prompts = parser.text2prompts(retrieved_text)

msg = ava.request([
    *text_prompts,
        {
        "type": "text",
        "text": query
    }
])

pprint(len(retrieved_docs))
pprint(msg)

25
Hier sind die extrahierten Informationen zu den angeforderten Themen aus dem LaTeX-Dokument:

### Art der baulichen Nutzung

#### Eingeschränktes Gewerbegebiet (GEE) (§ 8 BauNVO i.V.m. § 1 (4) BauNVO)
- **Zulässig sind / zulässig ist:**
  - Gewerbebetriebe, die das Wohnen nicht wesentlich stören, wobei bei Einzelhandelsbetrieben (gem. § 1 (5) BauNVO i.V.m. § 1 (9) BauNVO) nur die nicht innenstadtrelevanten Sortimente Kfz, Motorräder, Mopeds, Fahrräder, Kfz-Zubehör, Rasenmäher, Landmaschinen, Fahrrad- und Motorradzubehör, Brennstoffe und Mineralölerzeugnisse zulässig sind.
  - Der Verkauf von auf dem Grundstück produzierten Waren auf einer untergeordneten Fläche ist zulässig.
- **Nicht zulässig sind / nicht zulässig ist:**
  - Nutzungen gemäß § 8 (3) Nr. 3 BauNVO (Vergnügungsstätten) sind gemäß § 1 (6) BauNVO nicht Bestandteil des Bebauungsplans und damit nicht zulässig.
  - Einzelhandelsbetriebe mit innenstadtrelevanten und bestimmten nicht innenstadtrelevanten Sortimenten sind gemä